# ETF Holdings Downloader

## Introduction

Exchange-Traded Funds (ETFs) are investment funds traded on stock exchanges, much like stocks. They hold assets such as stocks, commodities, or bonds and generally operate with an arbitrage mechanism designed to keep trading close to its net asset value, though deviations can occasionally occur. ETFs provide investors with a way to diversify their portfolios without having to buy individual assets.

Holdings refer to the individual assets that an ETF owns. Knowing the holdings of an ETF can help investors understand the exposure and risk associated with the ETF.

## Project Overview

This project is designed to automate the process of downloading and processing the holdings of a list of ETFs identified by their ISINs (International Securities Identification Numbers). The script performs the following tasks:

1. **Download Holdings Data**: For each ISIN in the provided list, the script downloads the holdings data from a specified URL.
2. **Process Data**: The downloaded data is processed to:
   - Remove unnecessary rows and columns.
   - Rename columns for consistency.
   - Convert asset percentages to a more readable format.
   - Insert the file name for reference.
3. **Save Processed Data**: The processed data is saved as CSV files with a specified separator.

## Usefulness

This project is useful for investors, analysts, and financial professionals who need to regularly update and analyze the holdings of multiple ETFs. By automating the download and processing of holdings data, it saves time and reduces the potential for manual errors. The processed data can then be used for further analysis, reporting, or integration into other financial models and tools.

In [ ]:
import os
import requests
import pandas as pd
from io import BytesIO

In [ ]:
# Creting list of necessary ISINs
isin_list = ["IE00BDR5HM97" , "IE00BL25JM42" , "IE00BJ0KDQ92" , "IE00BL25JL35" , "IE00BM67HK77" , 
             "IE00BZ02LR44" , "IE00BJ0KDR00" , "IE00BFMNPS42" , "IE00BJZ2DC62" , "IE00BG36TC12" , 
             "IE00BDGN9Z19" , "IE00BTJRMP35" , "IE00BG370F43" , "IE00BD4DXB77" , "IE00BM67HQ30" , 
             "IE00BNC1G699" , "IE00BJZ2DD79" , "IE00BLNMYC90" , "IE00BGV5VN51"]

# Base URL for the Xtrackers ETFs page
base_url = 'https://etf.dws.com/etfdata/export/LUX/ENG/excel/product/constituent/'

# Destination directory for saving files
save_dir = 'C:\\Users\\ftaranta\\OneDrive - ICE Inc\\Desktop\\PythonScriptFilippo\\Script prove\\holding\\holding_xtrackers'

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Loop through the ISINs
for isin in isin_list:
    try:
        # Create the full URL by replacing the ISIN in the base URL
        url = f'{base_url}{isin}/'
        
        # Download the file
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Read the Excel file into a DataFrame
            df = pd.read_excel(BytesIO(response.content), engine='openpyxl')
            
            file_path = os.path.join(save_dir, f'{isin}.csv')
            
            # Save df as CSV
            df.to_csv(file_path, index=False, sep="|")
    except Exception as e:
        print(f'Error during the process for {isin}: {e}')


In [ ]:
# Working on all files in the directory
for file in os.listdir(save_dir):
    if file.endswith(".csv"):
        file_path = os.path.join(save_dir, file)
        
        # Sicne the first rows are uselees to our analysis, read the CSV file and skip the first 3 rows
        df = pd.read_csv(file_path, skiprows=3, encoding="utf-8-sig", sep = "|")

        # Rename columns
        df.rename(columns={
            "ISIN": "underlying_isin",
            "Country": "location",
            "Currency": "currency",
            "Type of Security": "asset_class",
            "Name": "underlying_name",
            "Industry Classification": "sector",
            "Weighting": "asset_perc"
        }, inplace=True)

        # In the original file, the percentage is expressed as a decimal, so we multiply it by 100
        df['asset_perc'] = df['asset_perc'] * 100
        df.loc[:, 'asset_perc'] = df['asset_perc'].fillna(0)
        df.drop('Unnamed: 0', axis=1, inplace=True)

        # Droppa columns if they exist
        columns_to_drop = ['Rating', 'Primary Listing', 'Exchange']
        for column in columns_to_drop:
            if column in df.columns:
                df.drop(columns=[column], inplace=True)

        df.insert(0, 'file_name', file)

        # Save the DataFrame as CSV with ";" separator
        df.to_csv(file_path, index=False, sep=";")
        
        print(f'File {file} processato e salvato.')

print('Elaborazione completata.')


